In [3]:
import transformers
from tqdm import tqdm 
import pandas as pd
import json
import os

## resources for SBERT
https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark.py 
https://www.sbert.net/examples/training/sts/README.html#loss-function 
https://github.com/cerlymarco/MEDIUM_NoteBook/blob/master/Siamese_Dual_BERT/Siamese_Dual_BERT.ipynb
    

## start by getting the data in the right format for training 

In [8]:
outerFolder = "/shared/3/projects/newsDiffusion/data/raw/train"

dfList = []
for folderNum in tqdm(os.listdir(outerFolder)): 
    innerFolder = outerFolder + "/" + folderNum
    for innerFileName in os.listdir(innerFolder): 
        innerPathName = outerFolder + "/" + folderNum + "/" + innerFileName
        if "json" in innerPathName: 
            with open(innerPathName, "r") as f: 
                data = json.loads(f.read())
            df = pd.json_normalize(data)
            df["id"] = innerFileName.split(".")[0]
            dfList.append(df)
        

100%|██████████| 100/100 [00:43<00:00,  2.32it/s]


In [9]:
fullDf = pd.concat(dfList).reset_index(drop=True)

In [1]:
leanDf = fullDf[["id", "title", "url", "source_url", "text"]]

NameError: name 'fullDf' is not defined

In [24]:
groundTruths = pd.read_csv("/shared/3/projects/benlitterer/agendaSetting/trainingDataV2.csv")
print(groundTruths.shape)
groundTruths.head(3)

(4964, 14)


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,4.0,1.0,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,4.0,1.0,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-...,https://www.timesofisrael.com/trump-says-he-do...,https://web.archive.org/web/www.teaparty.org/t...,https://web.archive.org/web/www.timesofisrael....,1.0,2.0,1.0,2.333333,2.333333,1.000000,1.333333


In [25]:
groundTruths[["id1", "id2"]] = groundTruths["pair_id"].str.split("_", expand=True)

In [36]:
leanDf = leanDf[["id", "text", "title"]]
leanDf.head(3)

,id,text,title
0,1483981361,Bottrop-Kirchhellen. Zum Jahresausklang ist de...,Bottroper Läufer gehen für guten Zweck auf die...
1,1484189061,Montana Jewish Community Is A Model For Others...,Montana Jewish Community Is A Model For Others...
2,1544781061,People who have contracted coronavirus are at ...,Coronavirus could be most contagious before sy...


In [67]:
len(groundTruths)

4964

In [68]:
#merge text for the first id, id1
leanDf1 = leanDf.rename(columns = {"id":"id1","text":"text1", "title":"title1"})
firstMerge = pd.merge(groundTruths, leanDf1, how="left", on="id1")
firstMerge.head(3)

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone,id1,id2,text1,title1
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,4.0,1.0,4.000000,4.000000,1.666667,2.000000,1484084337,1484110209,"MARTINSBURG, W.Va. — A suspected drunken drive...",Virginia man arrested in fatal DUI crash in We...
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,4.0,1.0,4.000000,3.666667,1.666667,1.333333,1484396422,1483924666,Share This On:\n\nPin 11 Shares\n\n(NEWS ROOM ...,Guyana: Three injured after car crashes into u...
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-...,https://www.timesofisrael.com/trump-says-he-do...,https://web.archive.org/web/www.teaparty.org/t...,https://web.archive.org/web/www.timesofisrael....,1.0,2.0,1.0,2.333333,2.333333,1.000000,1.333333,1484698254,1483758694,(Breitbart) – President Donald Trump welcomed ...,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...


In [69]:
leanDf2 = leanDf.rename(columns = {"id":"id2","text":"text2", "title":"title2"})
merged = pd.merge(firstMerge, leanDf2, how="left", on="id2")
merged.head(3)

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone,id1,id2,text1,title1,text2,title2
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,4.0,1.0,4.000000,4.000000,1.666667,2.000000,1484084337,1484110209,"MARTINSBURG, W.Va. — A suspected drunken drive...",Virginia man arrested in fatal DUI crash in We...,"PORT-AU-PRINCE, Haiti — Haitian President Jove...",Haiti’s leader marks independence day amid sec...
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,4.0,1.0,4.000000,3.666667,1.666667,1.333333,1484396422,1483924666,Share This On:\n\nPin 11 Shares\n\n(NEWS ROOM ...,Guyana: Three injured after car crashes into u...,BERLIN - A fire at a zoo in western Germany in...,Fire kills more than 30 animals at zoo in west...
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-...,https://www.timesofisrael.com/trump-says-he-do...,https://web.archive.org/web/www.teaparty.org/t...,https://web.archive.org/web/www.timesofisrael....,1.0,2.0,1.0,2.333333,2.333333,1.000000,1.333333,1484698254,1483758694,(Breitbart) – President Donald Trump welcomed ...,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,"PALM BEACH, United States — US President Donal...","Trump says he does not expect war with Iran, ‘..."


In [70]:
#NOTE: when we do two inner joins above, we lose 14 articles, going down 
#to 4,951 total articles

In [71]:
#drop all data where both the title and text is na for either article pair 
mergedClean = merged.dropna(subset=["title1", "text1"], how="all").dropna(subset=["title2", "text2"], how="all")

In [72]:
#all of the common articles : 
removed = list(set(merged["pair_id"]) - set(mergedClean["pair_id"]))

removedDf = merged[merged["pair_id"].isin(removed)]

In [73]:
removedDf.head(5)

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone,id1,id2,text1,title1,text2,title2
264,en,en,1484007787_1484063147,http://rssfeeds.kare11.com/~/614376126/0/kare1...,https://www.usnews.com/news/best-states/minnes...,https://web.archive.org/web/rssfeeds.kare11.co...,https://web.archive.org/web/www.usnews.com/new...,1.00,1.00,1.00,2.00,2.00,1.00,1.00,1484007787,1484063147,NaN,NaN,MINNEAPOLIS (AP) — Police say one man is dead ...,"1 Man Killed, Another Injured in North Minneap..."
311,en,en,1484036494_1484370272,https://www.cbc.ca/news/canada/toronto/407-tol...,http://feedproxy.google.com/~r/bestnaira/BTbw/...,https://web.archive.org/web/www.cbc.ca/news/ca...,https://web.archive.org/web/feedproxy.google.c...,4.00,4.00,1.00,2.00,4.00,1.00,1.00,1484036494,1484370272,The price of travelling the 407 ETR through th...,,NaN,NaN
350,en,en,1484003043_1484127233,https://www.voanews.com/episode/kenya-has-peri...,https://www.standardmedia.co.ke/ktnnews/video/...,https://web.archive.org/web/www.voanews.com/ep...,https://web.archive.org/web/www.standardmedia....,1.75,3.75,1.25,3.75,3.25,1.75,1.75,1484003043,1484127233,The world has a plastic pollution problem. Tha...,Kenya Has a Perilous Plastic Pollution Problem,NaN,NaN
431,en,en,1484037777_1484370272,https://www.staradvertiser.com/2019/12/31/hawa...,http://feedproxy.google.com/~r/bestnaira/BTbw/...,https://web.archive.org/web/www.staradvertiser...,https://web.archive.org/web/feedproxy.google.c...,4.00,4.00,1.00,4.00,4.00,2.00,2.50,1484037777,1484370272,There were 272 record high temperatures and ti...,"More than 270 record high temperatures, ties r...",NaN,NaN
997,en,en,1484011774_1484200356,https://books.google.com/books?id=UMfv38IQtr0C...,https://www.ksdk.com/article/features/st-louis...,https://web.archive.org/web/books.google.com/b...,https://web.archive.org/web/www.ksdk.com/artic...,4.00,4.00,4.00,4.00,4.00,1.00,1.00,1484011774,1484200356,A NEW YORK TIMES NOTABLE BOOK • The definitive...,Thurgood Marshall,NaN,NaN


In [80]:
merged.loc[merged["pair_id"] == "1484007787_1484063147", "link1"]

264    http://rssfeeds.kare11.com/~/614376126/0/kare1...
Name: link1, dtype: object

In [81]:
len(mergedClean)

4951

In [82]:
len(mergedClean[(mergedClean["url1_lang"] == "en") & (mergedClean["url2_lang"] == "en")])

1791

In [83]:
#do some more thoughtful dropping of na's 
#this time for empty text and title fields 
mergedClean[["title1", "title2", "text1", "text2"]] = mergedClean[["title1", "title2", "text1", "text2"]].fillna("")

mergedClean["titleText1"] = mergedClean["title1"] + mergedClean["text1"]
mergedClean["titleText2"] = mergedClean["title2"] + mergedClean["text2"]

#NOTE: we lose 29 records here
#NOTE: we lose 10 records here according to the SemEval paper 
def splitLen(inStr): 
    return len(inStr.split())

#we want to keep only the data for which we have over 2 tokens 
mergedClean = mergedClean[(mergedClean["titleText1"].apply(len) > 0) & (mergedClean["titleText2"].apply(len) > 0)]
len(mergedClean)

4950

In [84]:
mergedClean = mergedClean.drop(columns=["titleText1", "titleText2"])

In [114]:
#all of the common articles : 
removed = list(set(merged["pair_id"]) - set(mergedClean["pair_id"]))

removedDf = merged[merged["pair_id"].isin(removed)]

In [115]:
len(removed)

14

In [116]:
merged.loc[6]

url1_lang                                                   en
url2_lang                                                   en
pair_id                                  1484034982_1483785560
link1        https://www.financialexpress.com/market/commod...
link2        https://www.financialexpress.com/india-news/de...
ia_link1     https://web.archive.org/web/www.financialexpre...
ia_link2     https://web.archive.org/web/www.financialexpre...
Geography                                                  3.0
Entities                                                   4.0
Time                                                  1.333333
Narrative                                                  4.0
Overall                                               3.666667
Style                                                 1.333333
Tone                                                  1.333333
id1                                                 1484034982
id2                                                 148

In [121]:
shortRemoved

,pair_id,id1,id2,title1,text1,title2,text2
264,1484007787_1484063147,1484007787,1484063147,NaN,NaN,"1 Man Killed, Another Injured in North Minneap...",MINNEAPOLIS (AP) — Police say one man is dead ...
311,1484036494_1484370272,1484036494,1484370272,,The price of travelling the 407 ETR through th...,NaN,NaN
350,1484003043_1484127233,1484003043,1484127233,Kenya Has a Perilous Plastic Pollution Problem,The world has a plastic pollution problem. Tha...,NaN,NaN
431,1484037777_1484370272,1484037777,1484370272,"More than 270 record high temperatures, ties r...",There were 272 record high temperatures and ti...,NaN,NaN
997,1484011774_1484200356,1484011774,1484200356,Thurgood Marshall,A NEW YORK TIMES NOTABLE BOOK • The definitive...,NaN,NaN
1216,1483806200_1484043880,1483806200,1484043880,What to watch on Wednesday: ‘Spinning Out’ pre...,The Circle (Netflix) Status and strategy colli...,NaN,NaN
1392,1484010524_1483798636,1484010524,1483798636,NaN,NaN,"Yes, Anderson Cooper’s ‘cock’ story about his ...","When I was a kid, New Year’s Eve night was one..."
1515,1484759952_1484698242,1484759952,1484698242,NaN,NaN,"1 man dead, 2 others injured following fight i...",MINNEAPOLIS (AP) -- An altercation in downtown...
1886,1484209302_1484305743,1484209302,1484305743,NaN,NaN,Ezequiel Molina pide evitar desconfianza y des...,El pastor Ezequiel Molina pidió evitar formar ...
2482,1508535263_1523352839,1508535263,1523352839,NaN,NaN,ABC forced to pay costs of AFP legal fight,Federal Police raid ABC offices in Sydney over...


In [117]:
shortRemoved = removedDf[["pair_id", "id1", "id2", "title1", "text1", "title2", "text2"]]
removed1 = shortRemoved[["id1", "title1", "text1"]]
ids1 = list(removed1.loc[removed1["text1"].isna(), "id1"])

In [118]:
removed2 = shortRemoved[["id2", "title2", "text2"]]
removed2
ids2 = list(removed2.loc[removed2["text2"].isna(), "id2"])

In [119]:
#send data from all languages 
mergedClean.to_csv("/shared/3/projects/newsDiffusion/data/processed/allTrainData.tsv", sep="\t")

In [123]:
noTextId = "1514227906"

In [125]:
allProblemIds = ids1 + ids2 + [noTextId]

In [126]:
allProblemIds

['1484007787',
 '1484010524',
 '1484759952',
 '1484209302',
 '1508535263',
 '1622743303',
 '1484370272',
 '1484127233',
 '1484370272',
 '1484200356',
 '1484043880',
 '1505204186',
 '1545766920',
 '1514227906']

In [127]:
shortRemoved

,pair_id,id1,id2,title1,text1,title2,text2
264,1484007787_1484063147,1484007787,1484063147,NaN,NaN,"1 Man Killed, Another Injured in North Minneap...",MINNEAPOLIS (AP) — Police say one man is dead ...
311,1484036494_1484370272,1484036494,1484370272,,The price of travelling the 407 ETR through th...,NaN,NaN
350,1484003043_1484127233,1484003043,1484127233,Kenya Has a Perilous Plastic Pollution Problem,The world has a plastic pollution problem. Tha...,NaN,NaN
431,1484037777_1484370272,1484037777,1484370272,"More than 270 record high temperatures, ties r...",There were 272 record high temperatures and ti...,NaN,NaN
997,1484011774_1484200356,1484011774,1484200356,Thurgood Marshall,A NEW YORK TIMES NOTABLE BOOK • The definitive...,NaN,NaN
1216,1483806200_1484043880,1483806200,1484043880,What to watch on Wednesday: ‘Spinning Out’ pre...,The Circle (Netflix) Status and strategy colli...,NaN,NaN
1392,1484010524_1483798636,1484010524,1483798636,NaN,NaN,"Yes, Anderson Cooper’s ‘cock’ story about his ...","When I was a kid, New Year’s Eve night was one..."
1515,1484759952_1484698242,1484759952,1484698242,NaN,NaN,"1 man dead, 2 others injured following fight i...",MINNEAPOLIS (AP) -- An altercation in downtown...
1886,1484209302_1484305743,1484209302,1484305743,NaN,NaN,Ezequiel Molina pide evitar desconfianza y des...,El pastor Ezequiel Molina pidió evitar formar ...
2482,1508535263_1523352839,1508535263,1523352839,NaN,NaN,ABC forced to pay costs of AFP legal fight,Federal Police raid ABC offices in Sydney over...


In [64]:
mergedCleanEn = mergedClean[(mergedClean["url1_lang"] == "en") & (mergedClean["url2_lang"] == "en")]

In [65]:
len(mergedCleanEn)

1791

In [66]:
mergedCleanEn.to_csv("/shared/3/projects/newsDiffusion/data/processed/enTrainData.tsv", sep="\t")